In [1]:
# prepare packages
import requests, pandas as pd
from datetime import datetime, timedelta, timezone
from bs4 import BeautifulSoup
import logging
# from pythonjsonlogger import jsonlogger
import logmatic
import os
import time, random

In [2]:

# 設定現在時間 以時間來作為log檔名 一個log 以日計算
# 並在當下目錄創建 log folder
def get_time():
    now = datetime.utcnow()
    path = "./log"

    utc_8 = timezone(timedelta(hours=8))
    now = now.replace(tzinfo=timezone.utc)

    time_now = now.astimezone(utc_8)

    split = str(time_now).split(" ")
    file_date = split[0]
    split = split[1].split(":")
    file_time = split[0]+split[1]
    if not os.path.exists(path) :
        os.makedirs(path)
        
    return path + "/" + file_date 




In [3]:
# memory
#url = "https://www.ptt.cc/bbs/Food/index2.html"

# set proxy to use
proxylist = [{"http": "37.187.120.123:80"},
             {"https": "206.189.36.198:8080"},
             {"https": "178.128.31.153:8080"},
             {"http": "167.114.180.102:8080"},
             {"http": "104.131.214.218:80"},
             {"http": "167.114.196.153:80"}]
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}


In [4]:
# example use of logmatic 
# more simple 

logger = logging.getLogger()
 
handler = logging.FileHandler(get_time())
handler.setFormatter(logmatic.JsonFormatter())
 
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [5]:
# 製作 url List
def get_url(n=2,m=3):
    result = []
    # 增加直覺 從第n頁 到 第m頁
    m = m + 1
    for t in range(n,m):
        url = "https://www.ptt.cc/bbs/Food/index" + str(t) + ".html"
        result.append(url)
    return result

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  """Entry point for launching an IPython kernel.


In [6]:
# 連線流程
def get_connect(url):
    i = 0
    resp = ""
    try :
        resp = requests.get(url,headers=header,proxies=proxylist[i])
        resp.encoding = "utf-8"
        time.sleep(random.randint(1,4))
        # 設定連線狀況 log
        code = "HTTP response code = " + str(resp.status_code)
        logger.info(code,extra={"url" : url, "time_use" : resp.elapsed.total_seconds()})
        while resp.status_code != 200 :
            logger.info("Connect error",extra={"url" : url, "http response": resp.raise_for_status()})
            resp.close()
            i = (i + 1) % 6
            resp = requests.get(url,headers=header,proxies=proxylist[i])
            resp.encoding = "utf-8"



    except Exception as e:
        logger.error(e,exc_info=True)
    
    return resp.text 
    
    

In [13]:
def parse(urls):
# 進入ptt列表 抓取 標題、推文數、網址 
    result = []
    soup = BeautifulSoup(get_connect(urls))
    print("進入ptt : ",urls)
    post = soup.find_all("div", class_="r-ent")
    for r in post:
        # 取得推的數量
        push = 0
        if r.find("div", class_="nrec").text:
            try:
                push = int(r.find("div", class_="nrec").text)
            except ValueError:
                    pass
        if r.find("a"):
            # 取得網址
            href = "http://www.ptt.cc" + r.find("a")["href"]
            # 取得標題
            title = r.find("a").text

            # 進入第二層 進入每一篇
            # 去除公告 請益 為分類的文章
            if not "公告" in title :
                if not "請益" in title:
                    print("現在處理: ", href)
                    art = BeautifulSoup(get_connect(href))
                    article = []
                    content = art.find("div", id="main-content")
                    # 保存要丟棄的資訊
                    val = content.find_all("span", {"class":"article-meta-value"})
                    #  文章排版問題 有些文章沒有部份以下資訊
                    try:
                        author = val[0].text
                        title_in_content = val[2].text
                        date_in_content = val[3].text
                    except IndexError as e:
                        logger.warning("article span list out of index", extra = {"url" : href})
                        pass

                    # 開始丟棄資訊
                    removes = content.find_all("div", class_="article-metaline")
                    for remove in removes:
                        remove.extract()
                    removes = content.find_all("div", class_="article-metaline-right")
                    for remove in removes:
                        remove.extract()
                    removes = content.find_all("span", class_="f2")
                    for remove in removes:
                        remove.extract()
                    removes = content.find_all("div", class_="push")
                    for remove in removes:
                        remove.extract()
                    article.append({
                        "author" : author,
                        "title_in_content" : title_in_content,
                        "date_in_content" : date_in_content,
                        "content" : content.text
                    })

        result.append({
            "title" : title,
            "push" : push,
            "href" : href,
            "content" : article

        })
        

    return result








In [8]:
from concurrent.futures import ThreadPoolExecutor, wait, ProcessPoolExecutor, as_completed

with ProcessPoolExecutor(max_workers= 2) as executor:
    future_result = {executor.submit(parse, url) : url for url in get_url(5,6)}
    
    new_result = []
    
    for future in as_completed(future_result):
        new_result.append(future.result())
        print("work has done")
    

進入ptt :  https://www.ptt.cc/bbs/Food/index5.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1079547520.A.35E.html
進入ptt :  https://www.ptt.cc/bbs/Food/index6.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1083830551.A.BA5.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1083844000.A.C1A.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1079567406.A.F5A.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1083986656.A.79E.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1079689943.A.9DE.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1084061525.A.AA2.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1079708431.A.BBA.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1079738475.A.15E.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1084230361.A.0B0.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1079866429.A.E5A.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1084231477.A.EAE.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1079876659.A.5CB.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1084232438.A.0E5.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1079987492.A.F01.html
現在處理:  http://www.ptt.cc/bbs/Foo

In [12]:
for r in new_result:
    for er in r:
        for author in er["content"]:
            print(author["date_in_content"])

Thu May  6 08:45:55 2004
Thu May  6 11:55:55 2004
Sat May  8 03:33:36 2004
Sun May  9 00:42:56 2004
Mon May 10 23:21:04 2004
Mon May 10 23:32:42 2004
Mon May 10 23:41:19 2004
Tue May 11 10:08:44 2004
Sat May 15 00:46:03 2004
Sat May 15 02:50:29 2004
Fri May 21 10:52:52 2004
Sat May 22 16:56:04 2004
Sun May 23 20:11:45 2004
Tue May 25 18:32:07 2004
Wed May 26 01:05:41 2004
Wed May 26 23:53:39 2004
Fri May 28 00:04:27 2004
Fri May 28 03:07:18 2004
Sat May 29 16:28:23 2004
Sat May 29 16:28:23 2004
Wed Mar 17 18:50:23 2004
Thu Mar 18 00:10:14 2004
Fri Mar 19 10:11:11 2004
Fri Mar 19 15:09:56 2004
Fri Mar 19 23:22:32 2004
Sun Mar 21 10:55:54 2004
Sun Mar 21 13:58:01 2004
Mon Mar 22 20:43:59 2004
Tue Mar 23 21:34:58 2004
Wed Mar 24 00:05:10 2004
Wed Mar 24 13:32:30 2004
Wed Mar 24 15:22:05 2004
Wed Mar 24 15:22:05 2004
Thu Mar 25 19:11:48 2004
Thu Apr  1 00:07:25 2004
Mon Apr  5 16:27:22 2004
Tue Apr  6 19:35:59 2004
Sun Apr 18 22:26:57 2004
Thu Apr 29 12:45:55 2004
Tue May  4 02:05:56 2004


In [ ]:
my_list = parse('https://www.ptt.cc/bbs/Food/index5.html')

In [ ]:
# 進入ptt列表 抓取 標題、推文數、網址 
result = []
soup = BeautifulSoup(get_connect(url))
print("進入ptt : ",url)
post = soup.find_all("div", class_="r-ent")
for r in post:
            # 取得推的數量
    push = 0
    if r.find("div", class_="nrec").text:
        try:
            push = int(r.find("div", class_="nrec").text)
        except ValueError:
                pass
    if r.find("a"):
        # 取得網址
        href = "http://www.ptt.cc" + r.find("a")["href"]
        # 取得標題
        title = r.find("a").text
        
        # 進入第二層 進入每一篇
        # 去除公告 請益 為分類的文章
        if not "公告" in title :
            if not "請益" in title:
                print("現在處理: ", href)
                art = BeautifulSoup(get_connect(href))
                article = []
                content = art.find("div", id="main-content")
                # 保存要丟棄的資訊
                val = content.find_all("span", {"class":"article-meta-value"})
                author = val[0].text
                title_in_content = val[2].text
                date_in_content = val[3].text
                
                # 開始丟棄資訊
                removes = content.find_all("div", class_="article-metaline")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("div", class_="article-metaline-right")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("span", class_="f2")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("div", class_="push")
                for remove in removes:
                    remove.extract()
                article.append({
                    "author" : author,
                    "title_in_content" : title_in_content,
                    "date_in_content" : date_in_content,
                    "content" : content.text
                })
                
    result.append({
        "title" : title,
        "push" : push,
        "href" : href,
        "content" : article
        
    })
    
    time.sleep(random.randint(1,6))
                






In [ ]:
import json
# 轉成 Dataframe
df = pd.DataFrame(result)



In [ ]:
# 排列欄位順序

df = df[ ["title", "push", "href", "content" ] ]

In [ ]:
df.iloc[1]

In [ ]:
# I/O 較多 使用 thread
start = time.time()
with ThreadPoolExecutor(max_workers=4) as executor:
    results = executor.map(int, ['1', '2', '3', '4', '5'])
    for v in results:
        print(v)
end = time.time()
print("time used :",end - start)

In [ ]:
# example use of json_logging
# more detail
import logging
import sys

import json_logging

# log is initialized without a web framework name
json_logging.ENABLE_JSON_LOGGING = True
json_logging.init()

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

#logger.info("test log statement")
#logger.info("test log statement", extra={'props': {"extra_property": 'extra_value'}})

In [ ]:
# set logging 
# 設定 log config 
# less detail

handler = logging.FileHandler(get_time())  # Or FileHandler or anything else
# Configure the fields to include in the JSON output. message is the main log string itself
format_str = '%(message)%(levelname)%(name)%(asctime)'
formatter = jsonlogger.JsonFormatter(format_str)
handler.setFormatter(formatter)
logger = logging.getLogger(__name__)
logger.addHandler(handler)
logger.setLevel(logging.INFO)
# Normally we would attach the handler to the root logger, and this would be unnecessary
logger.propagate = False
